In [4]:
from pandas import concat
from pandas import DataFrame
# create sequence
length = 10
sequence = [i/float(length) for i in range(length)]
# create X/y pairs
df = DataFrame(sequence)
df = concat([df, df.shift(1)], axis=1)
df.dropna(inplace=True)
# convert to LSTM friendly format
values = df.values
X, y = values[:, 0], values[:, 1]
X = X.reshape(len(X), 1, 1)
print(X.shape, y.shape)

(9, 1, 1) (9,)


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
# configure network
n_batch = len(X)
n_epoch = 1000
n_neurons = 10
# design network
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [15]:
# fit network
for i in range(n_epoch):
    model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
    model.reset_states()

1/1 [==============================] - 0s 6ms/step - loss: 0.0443


1/1 [==============================] - 0s 7ms/step - loss: 0.0253


1/1 [==============================] - 0s 6ms/step - loss: 0.0142


1/1 [==============================] - 0s 7ms/step - loss: 0.0047


1/1 [==============================] - 0s 6ms/step - loss: 7.2549e-04


1/1 [==============================] - 0s 7ms/step - loss: 1.6834e-04


1/1 [==============================] - 0s 6ms/step - loss: 1.3354e-04


1/1 [==============================] - 0s 6ms/step - loss: 1.2678e-04


1/1 [==============================] - 0s 7ms/step - loss: 1.2515e-04


In [16]:
# online forecast
for i in range(len(X)):
    testX, testy = X[i], y[i]
    testX = testX.reshape(1, 1, 1)
    yhat = model.predict(testX, batch_size=1)
    print('>Expected=%.1f, Predicted=%.1f' % (testy, yhat))

ValueError: in user code:

    C:\Users\Administrator\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\engine\training.py:1572 predict_function  *
        return step_function(self, iterator)
    C:\Users\Administrator\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\engine\training.py:1562 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Administrator\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Administrator\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\distribute\distribute_lib.py:2825 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Administrator\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\distribute\distribute_lib.py:3600 _call_for_each_replica
        r

The upper model gives the error becuase the shapes in input batch and the testing batch are not the same.
 ValueError: Input 0 is incompatible with layer sequential_5: expected shape=(9, None, 1), found shape=(1, 1, 1)

Solution 1: Online Learning (Batch Size = 1)
One solution to this problem is to fit the model using online learning.

This is where the batch size is set to a value of 1 and the network weights are updated after each training example.

This can have the effect of faster learning, but also adds instability to the learning process as the weights widely vary with each batch.

Nevertheless, this will allow us to make one-step forecasts on the problem. The only change required is setting n_batch to 1 as follows:

In [18]:
from pandas import DataFrame
from pandas import concat
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
# create sequence
length = 10
sequence = [i/float(length) for i in range(length)]
# create X/y pairs
df = DataFrame(sequence)
df = concat([df, df.shift(1)], axis=1)
df.dropna(inplace=True)
# convert to LSTM friendly format
values = df.values
X, y = values[:, 0], values[:, 1]
X = X.reshape(len(X), 1, 1)
# configure network
n_batch = 1
n_epoch = 1000
n_neurons = 10
# design network
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
# fit network
for i in range(n_epoch):
	model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
	model.reset_states()
# online forecast
for i in range(len(X)):
	testX, testy = X[i], y[i]
	testX = testX.reshape(1, 1, 1)
	yhat = model.predict(testX, batch_size=1)
	print('>Expected=%.1f, Predicted=%.1f' % (testy, yhat))

9/9 [==============================] - 0s 3ms/step - loss: 6.6317e-04


9/9 [==============================] - 0s 3ms/step - loss: 3.6265e-04


9/9 [==============================] - 0s 3ms/step - loss: 1.5500e-04


9/9 [==============================] - 0s 3ms/step - loss: 8.3149e-05


9/9 [==============================] - 0s 2ms/step - loss: 5.6451e-05


9/9 [==============================] - 0s 3ms/step - loss: 3.8718e-05


9/9 [==============================] - 0s 3ms/step - loss: 2.1745e-05


9/9 [==============================] - 0s 3ms/step - loss: 1.4193e-05


9/9 [==============================] - 0s 3ms/step - loss: 1.6143e-05
>Expected=0.0, Predicted=0.0
>Expected=0.1, Predicted=0.1
>Expected=0.2, Predicted=0.2
>Expected=0.3, Predicted=0.3
>Expected=0.4, Predicted=0.4
>Expected=0.5, Predicted=0.5
>Expected=0.6, Predicted=0.6
>Expected=0.7, Predicted=0.7
>Expected=0.8, Predicted=0.8
